# Calculate heatwave occurrances 

## Heatwave definition

Heatwaves are now defined as:

Tmin > 95percentile AND Tmax > 95percentile

For more than 2 consecutive days (i.e. total of 3 or more days).

This replaces the definition of only Tmin > 99percentile for more than 3 consecutive days (total of 4 or more days).

This is what is requested from the Lancet. To be honest it's not clear whether this produces a substantially 'better' indicator since all heatwave indicators are arbitrary in absence of covariate data (i.e. impact data). Furthermore we know that the health impacts are mediated by many other things, so in any case we are truely interested just in the trends i.e. demonstrating that there is a) more heatwaves and b) more exposure to heatwaves - this can be followed by local studies but (as always) the point is to present a general risk factor trend.


In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from scipy import stats

from tqdm.notebook import tqdm, tnrange
from dask.diagnostics import ProgressBar
from joblib import Parallel, delayed
import os
import sys

project_path = os.path.abspath(os.path.join('..', '..'))
if project_path not in sys.path:
    sys.path.insert(0, project_path)
    
from source.config import DATA_SRC, WEATHER_SRC
from source import heatwave_indices

xr.set_options(keep_attrs=True)

# Figure settings
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = (5,2.5)
plt.rcParams['figure.titlesize'] = 'medium'
plt.rcParams['axes.titlesize'] = 'medium'
plt.rcParams['savefig.bbox'] = 'tight'

In [3]:
MAP_PROJECTION = ccrs.EckertIII()

### Setup Paths

> NOTE: considered just adding the newest year each time instead of re-calculating the whole thing. HOWEVER in reality, the input data is still changing year to year, so far have needed to re-calculate anyway (e.g. change in resolution, change from ERAI to ERA5, in the future probably use ERA5-Land, etc). Although it seems like a cool idea to have a reproducible method where each year you just add one thing, in practice its better to have one 'frozen' output corresponding to each publication, so that it's easy to go back later to find data corresponding to specific results. Additionally, generating one file per year means you have a folder full of files that are harder to share, and the outputs are in the end pretty small (<50MB in Float32)}.

# TODO adopt ERA5 Land one day, very large data could be a challenge, might need to cut globe into smaller parts to skip the oceans

In [4]:
MAX_YEAR = 2023

REFERENCE_YEAR_START = 1986
REFERENCE_YEAR_END = 2005


TEMPERATURES_FOLDER = WEATHER_SRC / 'era5_0.25deg' / 'daily_temperature_summary'
CLIMATOLOGY_QUANTILES_FOLDER = WEATHER_SRC / 'era5_0.25deg' / 'quantiles'


RESULTS_FOLDER = DATA_SRC / 'lancet'/ 'heatwaves'
RESULTS_FOLDER.mkdir(exist_ok=True)

INTERMEDIATE_RESULTS_FOLDER = DATA_SRC / 'lancet'/ 'heatwaves'/ f'results_{MAX_YEAR+1}'
INTERMEDIATE_RESULTS_FOLDER.mkdir(exist_ok=True)


In [5]:
assert INTERMEDIATE_RESULTS_FOLDER.is_dir()
assert RESULTS_FOLDER.is_dir()

In [11]:
# quantiles_files = list(CLIMATOLOGY_QUANTILES.rglob('*.nc'))

In [10]:
temperature_files = [(year, TEMPERATURES_FOLDER / f'{year}_temperature_summary.nc') for year in range(2022, MAX_YEAR+1)]

## Load ERA5 reference temperature quantiles

Load both the tmin and tmax quatiles and place in a list

In [7]:
QUANTILES = [0.95]
QUANTILE = 0.95
t_var = 'tmin'
CLIMATOLOGY_QUANTILES = (CLIMATOLOGY_QUANTILES_FOLDER / 
                         f'daily_{t_var}_quantiles_{"_".join([str(int(100*q)) for q in QUANTILES])}_1986-2005.nc')
t_min_quantiles = xr.open_dataset(CLIMATOLOGY_QUANTILES)#
t_min_threshold = t_min_quantiles.sel(quantile=QUANTILE, drop=True, tolerance=0.001, method='nearest')

t_var = 'tmax'
CLIMATOLOGY_QUANTILES = (CLIMATOLOGY_QUANTILES_FOLDER / 
                         f'daily_{t_var}_quantiles_{"_".join([str(int(100*q)) for q in QUANTILES])}_1986-2005.nc')
t_max_quantiles = xr.open_dataset(CLIMATOLOGY_QUANTILES)#
t_max_threshold = t_max_quantiles.sel(quantile=QUANTILE, drop=True, tolerance=0.001, method='nearest')


t_var = 'tmean'
CLIMATOLOGY_QUANTILES = (CLIMATOLOGY_QUANTILES_FOLDER / 
                         f'daily_{t_var}_quantiles_{"_".join([str(int(100*q)) for q in QUANTILES])}_1986-2005.nc')

t_thresholds = [t_min_threshold.to_array().squeeze(), t_max_threshold.to_array().squeeze()]

In [8]:
cos_lat = np.cos(np.radians(t_min_threshold.latitude))

# Define calculation functions for counting heatwave occurances

Apply heatwave index function to selected vars using selected threshold days

In [11]:
def ds_for_year(year):
    ds = xr.open_dataset(TEMPERATURES_FOLDER / f'{year}_temperature_summary.nc')
    ds = ds.transpose('time','latitude','longitude')
    return ds
    

def apply_func_for_file(func, year, t_thresholds, t_var_names, days_threshold=2):
    ds = ds_for_year(year)
    
    datasets_year = [ds[name] for name in t_var_names]
    result = func(datasets_year, t_thresholds, days_threshold)
    
    # Add a year dimension matching the input file
    result = result.expand_dims(dim={'year': [year]})
    return year, result

def apply_func_and_save(func, year, output_folder, t_thresholds,  t_var_names=['tmin', 'tmax'], 
                        days_threshold=2, overwrite=False,
                        filename_pattern='indicator_{year}.nc'
                       ):
    output_file = output_folder / filename_pattern.format(year=year)
    if output_file.exists() is False and overwrite is False:
        year, result = apply_func_for_file(func, year, t_thresholds, t_var_names=t_var_names, days_threshold=days_threshold)
        result.to_netcdf(output_file)
        return f'Created {output_file}'
    else:
        return f'Skipped {output_file}, already exists'
    


# Calculate heatwave occurances


## Multi-threshold versions


### Heatwave number of days

In [13]:
out_folder = INTERMEDIATE_RESULTS_FOLDER / 'heatwave_days_era5'

out_folder.mkdir(exist_ok=True)

res = Parallel(n_jobs=6, verbose=3)(
    delayed(apply_func_and_save)(heatwave_indices.heatwaves_days_multi_threshold, year, out_folder, t_thresholds, ['t_min', 't_max'])
    for year, file in  temperature_files
)



[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   2 | elapsed:   30.8s remaining:    0.0s
[Parallel(n_jobs=6)]: Done   2 out of   2 | elapsed:   30.8s finished


### Heatwave Counts

In [21]:
out_folder = INTERMEDIATE_RESULTS_FOLDER / 'heatwave_counts_era5'

out_folder.mkdir(exist_ok=True)

res = Parallel(n_jobs=6, verbose=2)(
    delayed(apply_func_and_save)(heatwave_indices.heatwaves_counts_multi_threshold, year, out_folder, t_thresholds, ['t_min', 't_max'])
    for year, file in  temperature_files
)


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  3.1min
[Parallel(n_jobs=6)]: Done  44 out of  44 | elapsed:  4.3min finished
